# Importing Libraries

In [1]:
import json
import yaml
import os
import rasa
import nest_asyncio
import asyncio
import random
import pandas as pd

nest_asyncio.apply()

# Loading the dataset

In [2]:
def load_data(dataset_dir, load_dataset_path, no_intents=None, entities=True):
    with open(load_dataset_path, 'w') as yaml_file:
        rasa_version = "3.1"
        yaml_file.writelines(f"version: {rasa_version}")
        yaml_file.write("\n\n")
        yaml_file.writelines("nlu:\n")
        intents = os.listdir(dataset_dir)
        for intent_file_name in intents:
            intent_path = os.path.join(dataset_dir, intent_file_name)
            intent_name = intent_file_name[:-5]
            yaml_file.writelines(f"- intent: {intent_name}\n")
            yaml_file.writelines("  examples: |\n")
            with open(intent_path) as json_file:
                json_data = json.load(json_file)
                intent_examples = json_data[intent_name]
                if(no_intents==None):
                    no_intents=len(intent_examples)
                text_examples = []
                intent_examples = random.sample(intent_examples, no_intents)
                for data in intent_examples:
                    texts = data["data"]
                    example = ""
                    for text in texts:
                        word = text["text"]
                        if entities:
                            if "entity" in text.keys():
                                entity = text["entity"]
                                word = f"[{word}]({entity})"
                        example = example + word
                    yaml_file.writelines(f"    - {example}\n")
            yaml_file.writelines("\n")
            json_file.close()
    yaml_file.close()

In [3]:
train_dataset_dir = "./dataset/Train/"
val_dataset_dir = "./dataset/Validate/"

In [11]:
load_data(train_dataset_dir, "./model_data/train_20_data.yml", no_intents=20, entities=True)
load_data(train_dataset_dir, "./model_data/train_100_data.yml", no_intents=100, entities=True)
load_data(train_dataset_dir, "./model_data/train_200_data.yml", no_intents=200, entities=True)

load_data(val_dataset_dir, "./model_data/val_data_entities.yml", entities=True)
load_data(val_dataset_dir, "./model_data/val_data_no_entities.yml", entities=False)

In [4]:
load_data(train_dataset_dir, "./model_data/train_20_data.yml", no_intents=20, entities=True)
load_data(val_dataset_dir, "./model_data/val_data_entities.yml", entities=True)
load_data(val_dataset_dir, "./model_data/val_data_no_entities.yml", entities=False)

# Training model

In [6]:
from rasa import model_training

c:\Users\darsh\Desktop\Chat Bot\env\lib\site-packages\future\standard_library\__init__.py:65: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [8]:
rasa.model_training.train_nlu("./configs/diet-replace-mask.yml", "./model_data/train_20_data.yml", "./models/")


c:\Users\darsh\Desktop\Chat Bot\env\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/cond_1_grad/gradients/cond_1/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/cond_1_grad/gradients/cond_1/GatherV2_grad/Reshape:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradients/cond_1_grad/gradients/cond_1/GatherV2_grad/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
c:\Users\darsh\Desktop\Chat Bot\env\lib\site-packages\rasa\utils\tensorflow\model_data.py:774: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.concatenate(np.array(f)),
Epochs: 100%|███

Your Rasa model is trained and saved at 'models\nlu-20221217-195337-jet-airport.tar.gz'.


'models\\nlu-20221217-195337-jet-airport.tar.gz'

In [16]:
rasa.model_training.train_nlu("./configs/diet-heavy.yml", "./model_data/train_20_data.yml", "./models/")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at rasa/LaBSE.
If yo

Your Rasa model is trained and saved at 'models\nlu-20221217-201702-skinny-quail.tar.gz'.


'models\\nlu-20221217-201702-skinny-quail.tar.gz'

In [20]:
rasa.model_training.train_nlu("./configs/config-origin.yml", "./model_data/train_20_data.yml", "./models/")

c:\Users\darsh\Desktop\Chat Bot\env\lib\site-packages\rasa\shared\utils\io.py:98: UserWarning: The out of vocabulary token 'oov' was configured, but could not be found in any one of the NLU training examples. All unseen words will be ignored during prediction.
  More info at https://rasa.com/docs/rasa/components#countvectorsfeaturizer
c:\Users\darsh\Desktop\Chat Bot\env\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Your Rasa model is trained and saved at 'models\nlu-20221217-202735-hushed-atmosphere.tar.gz'.


'models\\nlu-20221217-202735-hushed-atmosphere.tar.gz'

In [21]:
rasa.model_training.train_nlu("./configs/config-mega-basic.yml", "./model_data/train_20_data.yml", "./models/")

c:\Users\darsh\Desktop\Chat Bot\env\lib\site-packages\rasa\shared\utils\io.py:98: UserWarning: Entity entity 'facility' has only 1 training examples! The minimum is 2, because of this the training may fail.
c:\Users\darsh\Desktop\Chat Bot\env\lib\site-packages\rasa\shared\utils\io.py:98: UserWarning: You have defined training data consisting of entity examples, but your NLU configuration does not include an entity extractor trained on your training data. To extract non-pretrained entities, add one of MitieEntityExtractor, CRFEntityExtractor, DIETClassifier to your configuration.
  More info at https://rasa.com/docs/rasa/components


Your Rasa model is trained and saved at 'models\nlu-20221217-202812-chilly-rectangle.tar.gz'.


'models\\nlu-20221217-202812-chilly-rectangle.tar.gz'

In [12]:
rasa.model_training.train_nlu("./configs/diet-light.yml", "./model_data/train_20_data.yml", "./models/")

c:\Users\darsh\Desktop\Chat Bot\env\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/cond_grad/gradients/cond/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/cond_grad/gradients/cond/GatherV2_grad/Reshape:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradients/cond_grad/gradients/cond/GatherV2_grad/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
Epochs: 100%|██████████| 20/20 [01:15<00:00,  3.79s/it, t_loss=9.71, i_acc=0.714, e_f1=0.831]


Your Rasa model is trained and saved at 'models\nlu-20221217-200520-offline-chain.tar.gz'.


'models\\nlu-20221217-200520-offline-chain.tar.gz'

In [15]:
rasa.model_training.train_nlu("./configs/diet-replace.yml", "./model_data/train_20_data.yml", "./models/")

Epochs: 100%|██████████| 100/100 [03:28<00:00,  2.09s/it, t_loss=3.41, i_acc=1, e_f1=0.928]       


Your Rasa model is trained and saved at 'models\nlu-20221217-201213-isomorphic-extraction.tar.gz'.


'models\\nlu-20221217-201213-isomorphic-extraction.tar.gz'

# Evaluating the models

In [20]:
from rasa.core.agent import Agent

model_name = "diet-heavy-20"

model_path = f"./models/{model_name}.gz"
nlu_agent = Agent.load(model_path=model_path)

def get_prediction(query):
    pred = asyncio.run(nlu_agent.parse_message(message_data=query))
    return pred

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at rasa/LaBSE.
If yo

In [21]:
import rasa.shared.nlu.training_data.loading as nlu_loading

val_data = nlu_loading.load_data("./model_data/val_data_no_entities.yml")
val_data = [m.as_dict() for m in val_data.intent_examples]

val_data_entity = nlu_loading.load_data("./model_data/val_data_entities.yml")
val_data_entity = [m.as_dict() for m in val_data_entity.entity_examples]

In [22]:
columns = ["Query", "Expected Entity", "Predicted Entity", "Confidence"]
result = pd.DataFrame([], columns=columns)

for i in range(len(val_data)):
    query = val_data[i]["text"]
    pred = get_prediction(query)
    pred = pred["entities"]
    expec = val_data_entity[i]["entities"]
    token_point = {}
    for pred_entity in pred:
        token_point[pred_entity["start"]] = {
            "Expected Entity": ""
        }
        token_point[pred_entity["start"]]["Predicted Entity"] = pred_entity["entity"]
        token_point[pred_entity["start"]]["Confidence"] = pred_entity["confidence_entity"]
    for expec_entity in expec:
        if expec_entity["start"] not in token_point:
            token_point[expec_entity["start"]] = {
                "Predicted Entity": "",
                "Confidence": 0,
            }
        token_point[expec_entity["start"]]["Expected Entity"] = expec_entity["entity"]

    for token in token_point.values():
        row= {}
        row["Query"] = query
        row["Expected Entity"] = token["Expected Entity"]
        row["Predicted Entity"] = token["Predicted Entity"]
        row["Confidence"] = token["Confidence"]
        result = pd.concat([result, pd.DataFrame(row, index=[0])], ignore_index=True)

result.to_csv(f"./logs/entity-{model_name}.csv", index=False)

In [ ]:
columns = ["Query", "Expected Intent", "Predicted Intent"]+[intent[:-5] for intent in os.listdir("./dataset/Train/")]
result = pd.DataFrame([], columns=columns)

for data in val_data:
    row = {}
    query = data["text"]
    expected_intent = data["intent"]
    row["Query"] = query
    row["Expected Intent"] = expected_intent
    pred = get_prediction(query)
    row["Predicted Intent"] = pred["intent"]["name"]
    for pred_intent in pred["intent_ranking"]:
        row[pred_intent["name"]] = pred_intent["confidence"]

    result = pd.concat([result, pd.DataFrame(row, index=[0])], ignore_index=True)

result.to_csv(f"./logs/intent-{model_name}.csv")